We start with the code for an MDP class in Assignment 5

In [1]:
type MDP 
    stateSize  :: Int
    actionSize :: Int
    bellmanUpdate! :: Function
    c :: Array{Float64, 2}
    P :: Array{ Array{Float64, 2}, 1}
    
    function MDP(c, P)
        (n, m) = size(c)
        
        if length(P) != m
            error("Number of transition matrices does not match the number of actions")
        end
        
        P_concatenated = vcat(P...)
        if size(P_concatenated) != (n*m, n)
            error("Size of transition and reward matrices are inconsistent")
        end
        
        is_square(Pi) = size(Pi) == (n,n)
        is_row_stochastic(Pi) = isapprox(sum(Pi,2), ones(n); atol=100*eps(Float64))
        is_stopping_action(Pi) = Pi - zero(Pi)
        
        for Pi in P
            if !is_square(Pi)
                error("Transition matrix is not a square matrix")
                print(size(Pi))
                print(n)
            elseif !(is_row_stochastic(Pi) || is_stopping_action(Pi))
                error("Transition matrix is not row stochastic")
            end
        end
        
        function update!(v, g, vOld; discount=1)
            Q = c + discount * reshape(P_concatenated * vOld, n, m)
            
            for x=1:n
                g[x], v[x] = 1, Q[x,1]
                for u=2:m
                    if Q[x,u] < v[x]
                        g[x], v[x] = u, Q[x,u]
                    end
                end
            end
        end
        
        new(n, m, update!, c, P)
    end
end

And define the function to evaluate a policy

In [2]:
function evaluate(m::MDP, g;
                  discount=1.0)
    n = size(m.c, 1)
    c = zeros(n)
    P = zeros(n,n)
        
    for x=1:n
        u = g[x]
        c[x] = m.c[x,u]
        P[x,:] = m.P[u][x,:]
    end
    
    return (eye(n)-discount*P)\c
end

evaluate (generic function with 1 method)

In [3]:
function spanNorm(x, y)
    # z = x - y
    # return max(z) - min(z)
    # Optimized code. This could have been done using @devec, but
    # I don't want to add a dependency just for one function.
    z = vec(x - y)
    max_z, min_z = -Inf, Inf
    for elem in z 
        if max_z < elem
            max_z = elem
        end
        if min_z > elem
            min_z = elem
        end
    end
    return max_z - min_z
end

spanNorm (generic function with 1 method)

## Value Iteration

In [4]:
  function valueIteration(model::MDP, initial_v;
                    discount   = 0.95,
                    iterations = 1_000,
                    tolerance  = 1e-4)

        scaledDiscount  = (discount < 1)? (1-discount)/discount : 1
        scaledTolerance = scaledDiscount * tolerance / 2

        update!(v, g, initial_v) = model.bellmanUpdate!(v, g, initial_v; discount=discount)
        
        v_next = zero(initial_v)
        g_next = zeros(Int, size(initial_v))
    
        v_previous = copy(initial_v)
    
        update!(v_next, g_next, v_previous)
        v_precision = spanNorm(v_next, v_previous)

        iterationCount  = 1;
        while (v_precision > scaledTolerance && iterationCount < iterations)
            iterationCount += 1

            copy!(v_previous, v_next)
            update!(v_next, g_next, v_previous)

            v_precision = spanNorm(v_next, v_previous)
        end

        if (v_precision > scaledTolerance)
            warn(@sprintf("Value iteration did not converge. 
                 Reached precision %e at iteration %d", 2*v_precision/scaledDiscount, iterationCount))
        else
            info(@sprintf("Reached precision %e at iteration %d", 2*v_precision/scaledDiscount, iterationCount))
        end

        # Renormalize v -- See Puterman 6.6.12 for details
        v_next .+= minimum(v_next - v_previous)/scaledDiscount

        return (v_next, g_next)
    end



valueIteration (generic function with 1 method)

## Policy Iteration

In [5]:
function policyIteration(model::MDP, initial_g; discount=0.9)
    evaluate_policy(g) = evaluate(model, g; discount=discount)
    update!(v, g, initial_v) = model.bellmanUpdate!(v, g, initial_v; discount=discount)
    
    g_next = copy(initial_g)
    v_next = evaluate_policy(g_next)
   
    g_previous = zero(initial_g)
    v_previous = zero(v_next)
    
    iterationCount = 1
    
    while g_next != g_previous
        iterationCount += 1
        g_previous[:] = g_next[:]
        v_previous[:] = v_next[:]
        update!(v_next, g_next, v_previous)
        v_next = evaluate_policy(g_next)
    end
    
    info(@sprintf("Policy Iteration converged in %d iterations", iterationCount))
    return (v_next, g_next)
    
end

policyIteration (generic function with 1 method)

## System Described in the Question

In [6]:
const rate        = [0.2 0.5 0.8]
const arrivalRate = 0.6

const serviceCost = [0 2 5]
const holdingCost = 2

const R = 5
const M = 8
const A = length(rate)

P = [ spzeros(Float64, M+1, M+1) for u = 1:A]
c = zeros(Float64, M+1, A)

# Initialize cost matrix
c[1,:] = 0 

for x = 1:(M+1)
    for u = 1:A
        if x == 1 
          c[x,u] = serviceCost[u] 
        else
          c[x,u] = (x-1) * holdingCost + serviceCost[u] - R*rate[u]
        end
    end
end

# Initialize Probability matrix
for x = 2:M
    for u = 1:A
        P[u][x, x-1] = (1 - arrivalRate) * rate[u]
        P[u][x, x]   = (1 - arrivalRate) * (1 - rate[u]) + arrivalRate * rate[u]
        P[u][x, x+1] = arrivalRate * ( 1 - rate[u])
    end
end

for u = 1:A
    P[u][1,1] = (1 - arrivalRate) 
    P[u][1,2] = arrivalRate

    P[u][M+1, M+1] = (1 - rate[u])
    P[u][M+1, M  ] = rate[u]
end

model = MDP(c,P)

MDP(9, 3, update!, [0.0 2.0 5.0; 1.0 1.5 3.0; … ; 13.0 13.5 15.0; 15.0 15.5 17.0], Array{Float64,2}[[0.4 0.6 … 0.0 0.0; 0.08 0.44 … 0.0 0.0; … ; 0.0 0.0 … 0.44 0.48; 0.0 0.0 … 0.2 0.8], [0.4 0.6 … 0.0 0.0; 0.2 0.5 … 0.0 0.0; … ; 0.0 0.0 … 0.5 0.3; 0.0 0.0 … 0.5 0.5], [0.4 0.6 … 0.0 0.0; 0.32 0.56 … 0.0 0.0; … ; 0.0 0.0 … 0.56 0.12; 0.0 0.0 … 0.8 0.2]])

In [7]:
v0 = zeros(Float64, M+1)
g0 = zeros(Int,M+1) + 1 

9-element Array{Int64,1}:
 1
 1
 1
 1
 1
 1
 1
 1
 1

In [8]:
(vVI,gVI) = valueIteration(model, v0; discount=0.9)

INFO: Reached precision 9.295876e-05 at iteration 82


([23.3027, 27.6181, 36.9201, 49.6144, 64.5903, 80.9988, 97.9016, 113.255, 120.175], [1, 3, 3, 3, 3, 3, 3, 3, 3])

In [9]:
(vPI,gPI) = policyIteration(model, g0; discount=0.9)

INFO: Policy Iteration converged in 4 iterations


([23.3027, 27.6181, 36.9201, 49.6144, 64.5903, 80.9988, 97.9017, 113.255, 120.175], [1, 3, 3, 3, 3, 3, 3, 3, 3])

In [10]:
using Plots

In [11]:
scatter(0:M, [vVI vPI], label=["VI", "PI"], legend = :bottomright)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 5 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 100 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 VI 
 
 
 
 
 PI

In [12]:
scatter(0:M, [gVI-1 gPI-1], label = ["V1", "PI"], legend = :bottomright)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 5 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 V1 
 
 
 
 
 PI